In [1]:
from transformers import UperNetForSemanticSegmentation, AutoImageProcessor
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from evaluate import load

import sys
sys.path.append('..')

from training.dataset import SemanticSegmentationDataset
from training.trainer import SegmenterModeltrainer

/home/a-ploskin/repos/TerraLabel/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = UperNetForSemanticSegmentation.from_pretrained("openmmlab/upernet-swin-large")
processor = AutoImageProcessor.from_pretrained("openmmlab/upernet-swin-large")

In [3]:
img_dir = '/home/a-ploskin/repos/TerraLabel/data/task_0/data'
masks_dir = '/home/a-ploskin/repos/TerraLabel/data/masks'

train_ds, eval_ds = SemanticSegmentationDataset.get_train_and_eval_datasets(
    processor, img_dir, masks_dir
)

In [4]:
train_dataloader = DataLoader(train_ds, batch_size=2, shuffle=True, drop_last=True)
eval_dataloader = DataLoader(eval_ds, batch_size=2, shuffle=True, drop_last=True)

In [ ]:
log_path = './logs'
writer = SummaryWriter(log_path)
metric = load("mean_iou")

In [ ]:
trainer = SegmenterModeltrainer(
    model=model,
    device='cuda:0',
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
    metric=metric,
    writer=writer
)

segmenter = trainer.train(50)

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0


/home/a-ploskin/repos/TerraLabel/venv/lib/python3.12/site-packages/datasets/features/image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
  0%|          | 1/200 [00:44<2:27:19, 44.42s/it]

Mean_iou: 0.19919912219435118
Loss: 5.647426971073808
Mean accuracy: 0.3073365038597727
Epoch: 1


  1%|          | 2/200 [01:28<2:26:18, 44.34s/it]

Mean_iou: 0.25449818265367163
Loss: 3.238612384631716
Mean accuracy: 0.4260960918649654
Epoch: 2


  2%|▏         | 3/200 [02:12<2:24:48, 44.10s/it]

Mean_iou: 0.24200985953421142
Loss: 2.3037638777288896
Mean accuracy: 0.3443411666390379
Epoch: 3


  2%|▏         | 3/200 [02:31<2:45:48, 50.50s/it]


KeyboardInterrupt: 

In [ ]:
from datetime import datetime


model.save_pretrained(f'./models/uppernet_{datetime.now()}')